In [73]:
!pip install pymongo
from pymongo import MongoClient
import pymongo

In [74]:
#before running the code, please read each code block's intructions carefully. 

#Once you open up MongoDB Compass, copy the connection string below to connect to Compass. 
#Download MongoDB Community Edition to host a local cluster: https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-windows/ 
#An easy tutorial for Mac users: https://medium.com/create-a-clocking-in-system-on-react/creating-a-local-mongodb-database-and-insert-a-document-c6a4a2102a22
#Once you have MongoDB, be sure you can establish a local connection to Compass using the connection string: "mongodb://localhost:27017/"

client = MongoClient("mongodb://localhost:27017/")

In [75]:
#Inserting corona-out-2's raw data

In [95]:
#Define the database "DBFinalProject-out-2" and the collection "tweets".

db = client["my_database"]
collection_name = db["tweets"]

In [77]:
#The following code opens the corona-out-2 file and loads it into "parsed_data and writes it onto disk.
#Expect this to take a while. 

#The first part loads the data. 
#Note that due to the nature of the original file, each line must be split as shown below. 
#You should modify the code below with the directory to your "corona-out-2" file

import json

with open('/Users/mazinrafi/Downloads/corona-out-2', 'r') as f:
    parsed_data = []
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        parsed_data.append(obj)

        
#This second part writes the parsed file as a json file to disk. This file is called corona-out-2.json
#You may modify the file's extension to where you would like. This is saved to my download files. 
with open('/Users/mazinrafi/Downloads/corona-out-2.json', 'w') as f:
    json.dump(parsed_data, f)

In [96]:
#This will load the json file as "file_data" and uses the MongoDB "insert_many" command to insert the tweets. 
#After running this, the db: DBFinalProject-out-2 and "tweets" collection should be viewable on MongoDB Compass. 

#First we read the newly created json file from disk. As expected, this should take a while. 

import json
with open('/Users/mazinrafi/Downloads/corona-out-2.json') as f:
    file_data = json.load(f)
#The total number of tweets is 18518 if you run collection_name.insert_many(file_data). But is that actually true?
    
    
#The actual number of tweets with a unique ID is 18506, meaning 12 documents don't
#have a document id associated with it
We see this here: 
for item in file_data:
    # Insert or update the document in the collection using the "id" field as the unique identifier
    collection_name.replace_one({'id': item['id']}, item, upsert=True)

In [97]:
#Inserting corona-out-3 raw data. Similar setup to above.

In [98]:
import json

with open('/Users/mazinrafi/Downloads/corona-out-3', 'r') as f:
    parsed_data = []
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        parsed_data.append(obj)

with open('/Users/mazinrafi/Downloads/corona-out-3.json', 'w') as f:
    json.dump(parsed_data, f)

In [105]:
#Note. Because there are more than 100K documents in this file, the load of this will be INCREDIBLY slow. 
#This hasn't been fully tested on the corona-out-3 document. Please be careful before doing this one. 

client = MongoClient("mongodb://localhost:27017/")

db = client["my_final_database"]
collection_name = db["tweets"]

import json
with open('/Users/mazinrafi/Downloads/corona-out-3.json') as f:
    file_data = json.load(f)
    
#Using a "collection_name.insert_many(file_data)" will yield 108594

#The actual number of tweets with a unique ID is 108594, meaning 2 docs don't have an id. We see this here in 
#MongoDB after the insert: 
for item in file_data:
    # Insert or update the document in the collection using the "id" field as the unique identifier
    collection_name.replace_one({'id': item['id']}, item, upsert=True)

In [106]:
##Inserting the Users into MySQL 

!pip install mysql-connector-python
from mysql.connector import (connection)
import json
import mysql.connector

In [107]:
#You can download MySQL here: https://dev.mysql.com/downloads/installer/
#Connect to SQL database "localhost" after installing it.
#Note that the user and password must be defined during installation and re-used here.
#the user is "root" and the password is "gXt,:RsU#U-ws:3"
import mysql.connector

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
    print("Connected to MySQL server.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))


Connected to MySQL server.


In [110]:
# create a new database. This database is called "new_database"

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
cursor = connection1.cursor()
cursor.execute("CREATE DATABASE new_database")
print("Database created successfully!")

Database created successfully!


In [114]:
#Create a Table for the Users. This table is called "users": 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
cursor.execute("CREATE TABLE users (id bigint PRIMARY KEY,name VARCHAR(255), screen_name VARCHAR(255), followers_count int)")
print("Table created successfully!")

Table created successfully!


In [115]:
#Insert the users into the Table
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()

used = set()

for user in users:
    sql = "INSERT INTO users (id, name, screen_name, followers_count) VALUES (" + str(user['id']) + ",'" + user['name'].replace("'", "''") + "','" + user['screen_name'] + "'," + str(user['followers_count']) + ")"
    if user['id'] not in used:
        used.add(user['id'])
    else:
        continue
    cursor.execute(sql)

connection1.commit()

In [118]:
#Print some of the users in the Table
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()

# select the database
cursor.execute("USE new_database")

# execute the SQL command to fetch the first 5 rows of the table
cursor.execute("SELECT * FROM users LIMIT 5")

# fetch the result
result = cursor.fetchall()

# print the result
print(result)

[(111783, 'Michel Bozgounov', 'optimiced', 1804), (651403, 'Brian who survived a powerline', 'BrianTRice', 1759), (764036, 'Fran Monroy Moret', 'fmonroy', 84535), (769696, 'Bexxi 🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f🇪🇺', 'bexxi', 1079), (782257, 'phyphor', 'phyphor', 760)]


In [121]:
#print number of users (likely to be incorrect. Need to verify. Number is 15505): 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()

# select the database
cursor.execute("USE new_database")

# execute the SQL command to count the number of rows in the table
cursor.execute("SELECT COUNT(*) FROM users")

# fetch the result
result = cursor.fetchone()

# print the number of users
print("Number of users in the table:", result[0])

# close the cursor and connection
cursor.close()
connection1.close()

Number of users in the table: 15505


In [122]:
#Directly from file

import json
with open('/Users/mazinrafi/Downloads/corona-out-2.json') as f:
    file_data = json.load(f)

unique_users = set()

for i in file_data:
    unique_users.add(i['user']['id'])

num_unique_users = len(unique_users)
print("Number of unique users:", num_unique_users)

Number of unique users: 15505


In [58]:
#For testing purposes if needed. Do not run the cell

#Drop DB. May need to adjust the connection accordingly. 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP DATABASE my_database"
cursor.execute(sql)

#Drop Table
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP TABLE users"
cursor.execute(sql)

#Test to see if your connection works: 

import mysql.connector
import json

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="corona_out_2_db")
    print("Connected to MySQL server and database.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))

In [ ]:
#Possible cache setup

import datetime
cache={}
CACHE_SIZE=10
def query(str):
    if str in cache:
        result,ttl=cache[str]
        cache[str]=(result,datetime.datetime.now())
        return result
    else:
        # ask the database about the result
        output=ask_database()
        if len(cache)>=CACHE_SIZE:
            query_to_replace=None
            min_time=None
            for query in cache:
                result,ttl=cache[query]
                if query_to_replace=None:
                    query_to_replace=query
                    min_time=ttl
                elif min_time>ttl:
                    query_to_replace=query
                    min_time=ttl
            cache.remove(query_to_replace)
        cache[str]=(output,datetime.datetime.now())
        return output

In [43]:
users

In [113]:
#Side Notes

#need to keep the Python
#JSON Import Utility
#Primary Key needed
#Database needed and ID
# create a new database. This database is called "new_database"

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
cursor.execute("DROP TABLE users")
print("Database created successfully!")


Database created successfully!
